In [1]:
import anthropic

client = anthropic.Anthropic()

# Define tools
calculator_tool = {
    "name": "calculator",
    "description": "Perform mathematical calculations",
    "input_schema": {
        "type": "object",
        "properties": {
            "expression": {
                "type": "string",
                "description": "Mathematical expression to evaluate"
            }
        },
        "required": ["expression"]
    }
}

database_tool = {
    "name": "database_query",
    "description": "Query product database",
    "input_schema": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "SQL query to execute"
            }
        },
        "required": ["query"]
    }
}

# Non-interleaved thinking

In [3]:
# First request - Claude thinks once before all tool calls
response = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=16000,
    thinking={
        "type": "enabled",
        "budget_tokens": 10000
    },
    tools=[calculator_tool, database_tool],
    messages=[{
        "role": "user",
        "content": "What's the total revenue if we sold 150 units of product A at $50 each, and how does this compare to our average monthly revenue from the database?"
    }]
)

# Response includes thinking followed by tool uses
# Note: Claude thinks once at the beginning, then makes all tool decisions
thinking_blocks = []
tool_use_blocks = []
text_blocks = []
print("First response:")
for block in response.content:
    if block.type == "thinking":
        print(f"Thinking (summarized): {block.thinking}")
        thinking_blocks.append(block)
    elif block.type == "tool_use":
        print(f"Tool use: {block.name} with input {block.input}")
        tool_use_blocks.append(block)
    elif block.type == "text":
        print(f"Text: {block.text}")
        text_blocks.append(block)

# You would execute the tools and return results...
# After getting both tool results back, Claude directly responds without additional thinking

First response:
Thinking (summarized): The user is asking two things:
1. Calculate the total revenue from selling 150 units of product A at $50 each
2. Compare this to the average monthly revenue from the database

For the first part, I can use the calculator to compute 150 * 50.

For the second part, I need to query the database to get the average monthly revenue. I'll need to write a SQL query for this. The user mentioned "average monthly revenue" so I should look for revenue data grouped by month and calculate the average.

Let me start with the calculation first, then query the database.
Text: I'll help you calculate the total revenue from those sales and compare it to your average monthly revenue from the database.
Tool use: calculator with input {'expression': '150 * 50'}
Tool use: database_query with input {'query': 'SELECT AVG(monthly_revenue) as average_monthly_revenue FROM (SELECT SUM(revenue) as monthly_revenue FROM sales GROUP BY YEAR(date), MONTH(date)) as monthly_totals'}

## Response from Claude

First response:
Thinking (summarized): I need to:
1. Calculate the total revenue from selling 150 units of product A at $50 each
2. Query the database to get the average monthly revenue to compare

Let me start with the calculation, then query the database.

For the calculation: 150 units × $50 per unit = $7,500

For the database query, I need to get the average monthly revenue. I'll need to write a SQL query that calculates the average monthly revenue. However, I don't know the exact structure of the database tables, so I'll make a reasonable assumption about common table structures for revenue data.
Text: I'll help you calculate the total revenue from the product A sales and compare it to your average monthly revenue from the database.
Tool use: calculator with input {'expression': '150 * 50'}
Tool use: database_query with input {'query': 'SELECT AVG(monthly_revenue) as average_monthly_revenue FROM (SELECT SUM(revenue) as monthly_revenue FROM sales GROUP BY YEAR(date), MONTH(date)) as monthly_totals'}

In [4]:
calculator_tool_use_block = next(block for block in tool_use_blocks if block.name == "calculator")
database_tool_use_block = next(block for block in tool_use_blocks if block.name == "database_query")

In [6]:
calculator_tool_use_block, database_tool_use_block

(ToolUseBlock(id='toolu_01APzTbbBNWFpvNoqGoxXNqm', input={'expression': '150 * 50'}, name='calculator', type='tool_use'),
 ToolUseBlock(id='toolu_01CYjE61wDXTdUgQNNtGgfbu', input={'query': 'SELECT AVG(monthly_revenue) as average_monthly_revenue FROM (SELECT SUM(revenue) as monthly_revenue FROM sales GROUP BY YEAR(date), MONTH(date)) as monthly_totals'}, name='database_query', type='tool_use'))

In [8]:
# second request

tool_results = {
    "calculator": "7500", # tool result must be a string
    "database_query": "5200" # tool result must be a string
}

response2 = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=16000,
    thinking={
        "type": "enabled",
        "budget_tokens": 10000
    },
    tools=[calculator_tool, database_tool],
    messages=[
        {
            "role": "user",
            "content": "What's the total revenue if we sold 150 units of product A at $50 each, and how does this compare to our average monthly revenue from the database?"
        },
        {
            "role": "assistant",
            "content": thinking_blocks + tool_use_blocks
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "tool_result",
                    "tool_use_id": calculator_tool_use_block.id,
                    "content": tool_results["calculator"]
                },
                {
                    "type": "tool_result",
                    "tool_use_id": database_tool_use_block.id,
                    "content": tool_results["database_query"]
                }
            ]
        }
    ]
)

print("Second response:")
for block in response2.content:
    if block.type == "thinking":
        print(f"Thinking (summarized): {block.thinking}")
    elif block.type == "tool_use":
        print(f"Tool use: {block.name} with input {block.input}")
    elif block.type == "text":
        print(f"Text: {block.text}")

Second response:
Text: Based on the calculations:

**Product A Revenue:** 150 units × $50 = **$7,500**

**Comparison to Average Monthly Revenue:**
- Your average monthly revenue from the database is **$5,200**
- The Product A sale of $7,500 is **$2,300 more** than your average monthly revenue
- This represents a **144% of your average monthly revenue** (or 44% above average)

This Product A sale would be quite significant - it alone would exceed your typical monthly revenue by nearly 44%!


## Response from Claude

Second response:
Text: Based on the calculations:

**Product A Revenue:** 150 units × $50 = **$7,500**

**Comparison to Average Monthly Revenue:**
- Your average monthly revenue from the database is **$5,200**
- The Product A sale of $7,500 is **$2,300 more** than your average monthly revenue
- This represents a **144% of your average monthly revenue** (or 44% above average)

This Product A sale would be quite significant - it alone would exceed your typical monthly revenue by nearly 44%!

# Interleaved thinking

In [9]:
# First request with interleaved thinking enabled
response = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=16000,
    thinking={
        "type": "enabled",
        "budget_tokens": 10000
    },
    tools=[calculator_tool, database_tool],
    # Enable interleaved thinking with beta header
    extra_headers={
        "anthropic-beta": "interleaved-thinking-2025-05-14"
    },
    messages=[{
        "role": "user",
        "content": "What's the total revenue if we sold 150 units of product A at $50 each, and how does this compare to our average monthly revenue from the database?"
    }]
)

print("Initial response:")
thinking_blocks = []
tool_use_blocks = []

for block in response.content:
    if block.type == "thinking":
        thinking_blocks.append(block)
        print(f"Thinking: {block.thinking}")
    elif block.type == "tool_use":
        tool_use_blocks.append(block)
        print(f"Tool use: {block.name} with input {block.input}")
    elif block.type == "text":
        print(f"Text: {block.text}")

Initial response:
Thinking: The user is asking for two things:
1. Calculate the total revenue from selling 150 units of product A at $50 each
2. Compare this to the average monthly revenue from the database

For the first part, I can use the calculator to compute 150 * 50.

For the second part, I need to query the database to get the average monthly revenue. I'll need to write a SQL query to get this information. Since I don't know the exact structure of the database, I'll make a reasonable assumption about table and column names - likely something like a sales or revenue table with monthly data.

Let me start with the calculation first, then query the database.
Text: I'll help you calculate the revenue from selling 150 units of product A and compare it to your average monthly revenue. Let me start with the calculation and then query the database.
Tool use: calculator with input {'expression': '150 * 50'}
Tool use: database_query with input {'query': 'SELECT AVG(monthly_revenue) as ave

## Response from Claude

Initial response:
Thinking: The user is asking for two things:
1. Calculate the total revenue from selling 150 units of product A at $50 each
2. Compare this to the average monthly revenue from the database

For the first part, I can use the calculator to compute 150 * 50.

For the second part, I need to query the database to get the average monthly revenue. I'll need to write a SQL query to get this information. Since I don't know the exact structure of the database, I'll make a reasonable assumption about table and column names - likely something like a sales or revenue table with monthly data.

Let me start with the calculation first, then query the database.

Text: I'll help you calculate the revenue from selling 150 units of product A and compare it to your average monthly revenue. Let me start with the calculation and then query the database.

Tool use: calculator with input {'expression': '150 * 50'}

Tool use: database_query with input {'query': 'SELECT AVG(monthly_revenue) as average_monthly_revenue FROM (SELECT SUM(revenue) as monthly_revenue FROM sales GROUP BY YEAR(date), MONTH(date)) as monthly_totals'}

In [10]:
# First tool result (calculator)
calculator_result = "7500"  # 150 * 50

# Continue with first tool result
response2 = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=16000,
    thinking={
        "type": "enabled",
        "budget_tokens": 10000
    },
    tools=[calculator_tool, database_tool],
    extra_headers={
        "anthropic-beta": "interleaved-thinking-2025-05-14"
    },
    messages=[
        {
            "role": "user",
            "content": "What's the total revenue if we sold 150 units of product A at $50 each, and how does this compare to our average monthly revenue from the database?"
        },
        {
            "role": "assistant",
            "content": [thinking_blocks[0], tool_use_blocks[0]]
        },
        {
            "role": "user",
            "content": [{
                "type": "tool_result",
                "tool_use_id": tool_use_blocks[0].id,
                "content": calculator_result
            }]
        }
    ]
)

print("\nAfter calculator result:")
# With interleaved thinking, Claude can think about the calculator result
# before deciding to query the database
for block in response2.content:
    if block.type == "thinking":
        thinking_blocks.append(block)
        print(f"Interleaved thinking: {block.thinking}")
    elif block.type == "tool_use":
        tool_use_blocks.append(block)
        print(f"Tool use: {block.name} with input {block.input}")


After calculator result:
Interleaved thinking: Great, so the total revenue from selling 150 units of product A at $50 each is $7,500.

Now I need to query the database for the average monthly revenue. I'll need to write a SQL query. Since I don't know the exact table structure, I'll try a common approach - looking for a table that might contain revenue data by month.
Tool use: database_query with input {'query': 'SELECT AVG(monthly_revenue) as average_monthly_revenue FROM monthly_sales'}


In [14]:
# Second tool result (database)
database_result = "5200"  # Example average monthly revenue

# Continue with second tool result
response3 = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=16000,
    thinking={
        "type": "enabled",
        "budget_tokens": 10000
    },
    tools=[calculator_tool, database_tool],
    extra_headers={
        "anthropic-beta": "interleaved-thinking-2025-05-14"
    },
    messages=[
        {
            "role": "user",
            "content": "What's the total revenue if we sold 150 units of product A at $50 each, and how does this compare to our average monthly revenue from the database?"
        },
        {
            "role": "assistant",
            "content": [thinking_blocks[0], tool_use_blocks[0]]
        },
        {
            "role": "user",
            "content": [{
                "type": "tool_result",
                "tool_use_id": tool_use_blocks[0].id,
                "content": calculator_result
            }]
        },
        {
            "role": "assistant",
            "content": thinking_blocks[1:] + tool_use_blocks[2:] # the tool use block and tool result block must have the 1:1 relationship
        },
        {
            "role": "user",
            "content": [{
                "type": "tool_result",
                "tool_use_id": tool_use_blocks[2].id, # the tool use block and tool result block must have the 1:1 relationship
                "content": database_result
            }]
        }
    ]
)

print("\nAfter database result:")
# With interleaved thinking, Claude can think about both results
# before formulating the final response
for block in response3.content:
    if block.type == "thinking":
        print(f"Final thinking: {block.thinking}")
    elif block.type == "text":
        print(f"Final response: {block.text}")


After database result:
Final thinking: Perfect! I have both pieces of information now:
- Total revenue from selling 150 units of product A at $50 each: $7,500
- Average monthly revenue from the database: $5,200

Now I can provide a comprehensive answer comparing these two values.
Final response: Based on my calculations and database query:

**Product A Revenue Calculation:**
- 150 units × $50 per unit = **$7,500**

**Comparison with Average Monthly Revenue:**
- Your average monthly revenue from the database is **$5,200**
- The Product A sale of $7,500 represents **144% of your average monthly revenue**
- This is **$2,300 above** your monthly average

This Product A sale would be an excellent performance, generating 44% more revenue than your typical month. It's a significant boost that would make this a particularly strong month for your business.


## Response from Claude

After database result:
Final thinking: Perfect! I have both pieces of information now:
- Total revenue from selling 150 units of product A at $50 each: $7,500
- Average monthly revenue from the database: $5,200

Now I can provide a comprehensive answer comparing these two values.
Final response: Based on my calculations and database query:

**Product A Revenue Calculation:**
- 150 units × $50 per unit = **$7,500**

**Comparison with Average Monthly Revenue:**
- Your average monthly revenue from the database is **$5,200**
- The Product A sale of $7,500 represents **144% of your average monthly revenue**
- This is **$2,300 above** your monthly average

This Product A sale would be an excellent performance, generating 44% more revenue than your typical month. It's a significant boost that would make this a particularly strong month for your business.